# Data Cleaning and Preparation

In [2]:
import numpy as np
import pandas as pd


## Data Transformation (continue)
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

## String Manipulation
Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple with
the string object’s **built-in methods**. 

For more complex pattern matching and text manipulations, **regular expressions** may be needed. 

**pandas** adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data,
additionally handling the annoyance of missing data.

### String Object Methods

In [30]:
# convert the string to a list using the ',' as separator
val = 'a,b,  guido'
val.split(",")

['a', 'b', '  guido']

In [33]:
# convert the string to a list using the ',' as separator and remove the extra spaces
pieces = [v.strip() for v in val.split(",")]
pieces

['a', 'b', 'guido']

In [34]:
# concatenate the parts back with separator '::'
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [37]:
# concatenate the parts back with separator '::'
"::".join(pieces)
# f"::{pieces}::"

'a::b::guido'

**check**: try the `in` operator and str methods `index`, `find`, `count` and `replace`

![](assets/built-in-str-methods.png)

### Regular Expressions
Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language.

The `re` module functions fall into three categories: **pattern matching**, **substitution**,
and **splitting**.

In [45]:
import re
text = "foo    bar\t baz  \tqux"
# https://pythex.org
print(text)


foo ,   bar	 baz  	qux


suppose we wanted to split a string with a variable number of whitespace characters
(tabs, spaces, and newlines). The regex describing one or more whitespace characters
is \s+:

In [44]:
# split the string depending on the whitespaces
# text.split(" ")
re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

In [46]:
# find all the string depending on the whitespaces
re.findall("\w+", text)

['foo', 'bar', 'baz', 'qux']

**Note:** Creating a regex object with `re.compile` is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles

In [53]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [55]:
# get a list of all emails in the text
# re.findall(pattern, text, flags=re.IGNORECASE)
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Relatedly, `sub` will return a new string with occurrences of the pattern replaced by the
a new string:


In [60]:
print(regex.sub("SECRET", text))
# print(re.sub("[A-Z]\w+", "Name", text))

Dave SECRET
Steve SECRET
Rob SECRET
Ryan SECRET



Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: *username*, *domain name*, and *domain suffix*.

In [61]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [62]:
# find all email in the text
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [65]:
# prefix each segment of the email with a suitable label
print(regex.sub(r"user: \1, company: \2, ext: \3", text))

Dave user: dave, company: google, ext: com
Steve user: steve, company: gmail, ext: com
Rob user: rob, company: gmail, ext: com
Ryan user: ryan, company: yahoo, ext: com



![](assets/re-methods.png)

### Vectorized String Functions in pandas

In [66]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s **str attribute**

In [69]:
# check if the email is gmail
mask = data.str.contains("gmail").fillna(False)
data[mask]

Steve    steve@gmail.com
Rob        rob@gmail.com
dtype: object

Regular expressions can be used, too, along with any re options like IGNORECASE

In [71]:
# using the pattern declared earlier, find all parts of each email
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [72]:
# use the match method to check if the field matches an email or not
data.str.match(pattern, flags=re.IGNORECASE)

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

![](assets/series-str-methods.png)

# Data Aggregation and Group Operations

## GroupBy Mechanics

the term split-apply-combine is used for describing group operations. 

- In the first stage of the process, data contained in a pandas object, whether a Series, Data‐Frame, or otherwise, is **split** into groups based on one or more keys that you provide.
- Once this is done, a function is **applied** to each group, producing a new value. 
- Finally, the results of all those function applications are **combined** into a result object. 

The form of the resulting object will usually depend on what’s being done to the data.

![](assets/group-aggregation.png)

In [73]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(0, 10, 7),
                   'data2' : np.random.randint(0, 10, 7)})
df

,key1,key2,data1,data2
0,a,one,5,5
1,a,two,2,8
2,b,one,0,9
3,b,two,7,5
4,a,one,7,8
5,b,two,0,5
6,a,one,9,2


In [74]:
# group data of column 'data1' by 'key1' then print the groups
result = df["data1"].groupby(df["key1"])
result.groups

{'a': [0, 1, 4, 6], 'b': [2, 3, 5]}

In [77]:
# calculate the mean in each group
result.mean()

key1
a    5.750000
b    2.333333
Name: data1, dtype: float64

In [78]:
# group data of column 'data1' by 'key1' and 'key2' then print the groups
result = df["data1"].groupby([df["key1"], df["key2"]])
result.groups

{('a', 'one'): [0, 4, 6], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3, 5]}

In [83]:
# calculate the mean in each group
temp = result.mean()
temp

key1  key2
a     one     7.0
      two     2.0
b     one     0.0
      two     3.5
Name: data1, dtype: float64

In [84]:
temp[('a', 'one')]

7.0

In [86]:
# unstack the result Series
temp = temp.unstack()
temp

key2,one,two
key1,,
a,7.0,2.0
b,0.0,3.5


In [87]:
# try some selection on the result DataFrame
temp.loc["a", "one"]

7.0

### Iterating Over Groups
The GroupBy object supports iteration, generating a sequence of **2-tuples** containing
the **group name** along with the **chunk of data**.

In [89]:
# group data of DataFrame 'data' by 'key1' then print each group name and data
result = df.groupby(df["key1"])
for name, group in result:
    print(name)
    print(group)
    print("-" * 25)


a
  key1 key2  data1  data2
0    a  one      5      5
1    a  two      2      8
4    a  one      7      8
6    a  one      9      2
-------------------------
b
  key1 key2  data1  data2
2    b  one      0      9
3    b  two      7      5
5    b  two      0      5
-------------------------


In [90]:
# group data of DataFrame 'data' by 'key1' and 'key2' then print each group name and data
result = df.groupby([df["key1"], df["key2"]])
for name, group in result:
    print(name)
    print(group)
    print("-" * 25)


('a', 'one')
  key1 key2  data1  data2
0    a  one      5      5
4    a  one      7      8
6    a  one      9      2
-------------------------
('a', 'two')
  key1 key2  data1  data2
1    a  two      2      8
-------------------------
('b', 'one')
  key1 key2  data1  data2
2    b  one      0      9
-------------------------
('b', 'two')
  key1 key2  data1  data2
3    b  two      7      5
5    b  two      0      5
-------------------------


In [94]:
# group data of DataFrame 'data' by 'key1' then convert it to a dictionary of DataFrames
result = dict(list(df.groupby(df["key1"])))
result["b"]

,key1,key2,data1,data2
2,b,one,0,9
3,b,two,7,5
5,b,two,0,5


In [96]:
# group the columns by its datatypes, then print the groups
result = df.groupby(df.dtypes, axis=1)
result.groups

{int32: ['data1', 'data2'], object: ['key1', 'key2']}

In [97]:
# loop through the groups and print it
result = df.groupby(df.dtypes, axis=1)
for name, group in result:
    print(name)
    print(group)
    print("-" * 25)

int32
   data1  data2
0      5      5
1      2      8
2      0      9
3      7      5
4      7      8
5      0      5
6      9      2
-------------------------
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
5    b  two
6    a  one
-------------------------


### Selecting a Column or Subset of Columns
Indexing a GroupBy object created from a DataFrame with a column name or array
of column names has the effect of column subsetting for aggregation. This means
that:
```python
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
are syntactic sugar for:
```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```


In [99]:
# group data of DataFrame 'data' by 'key1' then calculate the mean of column 'data2'
# df["data2"].groupby(df["key1"]).mean()
df.groupby("key1")["data2"].mean()

key1
a    5.750000
b    6.333333
Name: data2, dtype: float64

In [100]:
# How to get result as DataFrame GroupBy or Series GroupBy?
df.groupby("key1")[["data2"]].mean()

,data2
key1,
a,5.750000
b,6.333333


### Grouping with Dicts and Series

In [101]:
people = pd.DataFrame(np.random.randint(0, 10, (5,5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0,8,3,8,6
Steve,1,9,1,9,2
Wes,0,9,2,0,4
Jim,4,0,7,8,4
Travis,7,7,5,9,7


In [102]:
mapping = {'Joe': 'red', 'Steve': 'red', 'Wes': 'blue',
           'Jim': 'blue', 'Travis': 'red', 'Elon' : 'orange'}

In [107]:
# group and sum the scores of teams red and blue
people.groupby(mapping).sum()

,a,b,c,d,e
blue,4,9,9,8,8
red,8,24,9,26,15


In [105]:
# convert the dict to a Series, then group and count the scores of teams red and blue
s = pd.Series(mapping)
people.groupby(s).sum()

,a,b,c,d,e
blue,4,9,9,8,8
red,8,24,9,26,15


### Grouping with Functions

In [106]:
people.groupby(len).sum()

,a,b,c,d,e
3,4,17,12,16,14
5,1,9,1,9,2
6,7,7,5,9,7


## independent Practice:
- read the dataset `tips.csv`
- create a new column 'tip_pct', which is tip / total_bill
- replace the short-day name with the full-day name and convert it to upper-case
- calculate the average tip percent for smokers and non-smokers
- calculate the max and average tip percent for each time
- calculate the average tip percent for each day and time
- create dummy variables for the day and time columns


